# Connect Drive

In [ ]:
# Connect Google Drive
from google.colab import drive4

drive.mount('/content/drive/')

================================================================================================

Download dataset pada link dibawah ini

Link Dataset: https://drive.google.com/file/d/191mgRxzIvNGlEwk6fM-WRRfsYzg3yfZP/view?usp=sharing

================================================================================================

In [ ]:
# Melakukan pengekstrakan Dataset
!unzip -q /content/drive/MyDrive/Dataset/Project_Data_Science/new_data/road_damage_dataset.zip -d /content/dataset/

# Import Library

## Install library

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

## Import Library

In [ ]:
from collections import Counter
from ultralytics import YOLO
from PIL import Image
from google.colab import files
from IPython.display import Image, display
import os
import matplotlib.pyplot as plt
import random
import shutil
import pandas as pd
import seaborn as sns
sns.set(style='whitegrid')

# Exploratory Data Analysis

## DIstribusi Data

In [ ]:
def count_classes(label_dir):
    class_counts = Counter()

    for file in os.listdir(label_dir):

        if file.endswith(".txt"):

            with open(os.path.join(label_dir, file), 'r') as f:

                lines = f.readlines()

                for line in lines:

                    cls_id = int(line.strip().split()[0])

                    class_counts[cls_id] += 1

    return class_counts

label_dir = '/content/dataset/labels'

print("Label Class Count:", count_classes(label_dir))



# Hitung distribusi
class_counts = count_classes(label_dir)

class_ids = list(class_counts.keys())

counts = list(class_counts.values())



# Plot
plt.figure(figsize=(8, 5))

plt.bar(class_ids, counts, color='skyblue')

plt.xlabel('Class ID')

plt.ylabel('Jumlah Bounding Box')

plt.title('Distribusi Kelas Label YOLO')

plt.xticks(class_ids)

plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

In [ ]:
# Melihat ukuran data paling kecil(minimum)

dataset = '/content/dataset/images'


img_size = []

for filename in os.listdir(dataset):

  if filename.endswith(('.jpg', '.png', '.jpeg')):

    img_path = os.path.join(dataset, filename)

    try:
      with Image.open(img_path) as img:

        width, height = img.size

        size_total = width * height

        img_size.append({'filename':filename, 'width':width, 'height':height, 'size_total': size_total})

    except Exception as e:

      print(f'error reading {filename}: {e}')



df_img_size = pd.DataFrame(img_size)

smallest_img = df_img_size.loc[df_img_size['size_total'].idxmin()]

smallest_img_path = os.path.join(dataset, smallest_img['filename'])

print(f"Gambar dengan ukuran terkecil: {smallest_img['filename']}")

print(f"Ukuran: {smallest_img['width']} x {smallest_img['height']}")



with Image.open(smallest_img_path) as img:

  plt.figure(figsize=(10,10))

  plt.imshow(img)

  plt.title(smallest_img['filename'])

  plt.axis('off')

  plt.show()

In [ ]:
# Visualisasi Distribusi Ukuran Gambar
plt.figure(figsize=(12, 6))



# lebar gambar
plt.subplot(1, 2, 1)

sns.histplot(data=df_img_size, x='width', bins=30, kde=True)

plt.title('Distribusi Lebar Gambar')

plt.xlabel('Lebar (Pixel)')

plt.ylabel('Jumlah Gambar')




# tinggi gambar
plt.subplot(1, 2, 2)

sns.histplot(data=df_img_size, x='height', bins=30, kde=True)

plt.title('Distribusi Tinggi Gambar')

plt.xlabel('Tinggi (Pixel)')

plt.ylabel('Jumlah Gambar')

plt.tight_layout()

plt.show()

In [ ]:
# Visualisasi Distribusi Ukuran Gambar
df_img_size['size'] = df_img_size.apply(lambda row: f"{row['width']}x{row['height']}", axis=1)

size_counts = df_img_size['size'].value_counts()



# Plot
plt.figure(figsize=(10, 6))

sns.barplot(x=size_counts.index, y=size_counts.values)

plt.title('Distribusi Ukuran Gambar')

plt.xlabel('Ukuran (Lebar x Tinggi)')

plt.ylabel('Jumlah Gambar')

plt.xticks(rotation=45)

plt.tight_layout()

plt.show()

# Preprocessing

## Konfigurasi Data.yaml

In [ ]:
# Membuat Konfigurasi Data.yaml

# Baca isi classes pada dataset
with open('/content/dataset/classes.txt', 'r') as f:

    class_list = [line.strip() for line in f.readlines()]



# Buat isi file data.yaml
data_yaml = f"""
train: /content/dataset/images/train
val: /content/dataset/images/val

nc: {len(class_list)}
names: {class_list}
"""



# Simpan sebagai file data.yaml
with open('data.yaml', 'w') as f:

    f.write(data_yaml)

print("File data.yaml berhasil dibuat!")

File data.yaml berhasil dibuat!


## Setting Hyperparameter

In [ ]:
# Setting Hyperparameter

bhypa_yaml = """
lr0: 0.001
lrf: 0.001
momentum: 0.8
weight_decay: 0.0001
box: 0.05
cls: 0.7
dfl: 0.75
hsv_s: 0.7
hsv_v: 0.3
label_smoothing: 0.1
scale: 0.2
degrees: 5
translate: 0.1
fliplr: 0.5
mosaic: 0.5
mixup: 0.2
copy_paste: 0.2
"""


with open("bhypa.yaml", "w") as f:

    f.write(bhypa_yaml.strip())

print("File bhypa.yaml berhasil dibuat!")

File bhypa.yaml berhasil dibuat!


## Train Test Split Dataset

In [ ]:
# Train Test Split


images_path = '/content/dataset/images'

labels_path = '/content/dataset/labels'



# Path ke folder split
train_images = os.path.join(images_path, 'train')

val_images = os.path.join(images_path, 'val')

train_labels = os.path.join(labels_path, 'train')

val_labels = os.path.join(labels_path, 'val')




# Buat folder train/val jika belum ada
os.makedirs(train_images, exist_ok=True)

os.makedirs(val_images, exist_ok=True)

os.makedirs(train_labels, exist_ok=True)

os.makedirs(val_labels, exist_ok=True)





# Ambil hanya file gambar (bukan folder)
image_files = [

    f for f in os.listdir(images_path)

    if f.endswith(('.jpg', '.png')) and os.path.isfile(os.path.join(images_path, f))
]




# Acak dan split
random.shuffle(image_files)

split_ratio = 0.8 # 80%(train) : 20%(val)

split_point = int(len(image_files) * split_ratio)

train_files = image_files[:split_point]

val_files = image_files[split_point:]




# Fungsi copy
def copy_data(files_list, src_img, src_lbl, dst_img, dst_lbl):

    for file in files_list:

        base = os.path.splitext(file)[0]

        img_file = os.path.join(src_img, file)

        lbl_file = os.path.join(src_lbl, base + '.txt')

        if os.path.exists(img_file):

            shutil.copy(img_file, os.path.join(dst_img, file))

        if os.path.exists(lbl_file):

            shutil.copy(lbl_file, os.path.join(dst_lbl, base + '.txt'))





# Eksekusi copy
copy_data(train_files, images_path, labels_path, train_images, train_labels)

copy_data(val_files, images_path, labels_path, val_images, val_labels)

print(f"Split selesai! Total: {len(train_files)} train, {len(val_files)} val")

Split selesai! Total: 161 train, 41 val


# Train Model

In [ ]:
# Load pretrained model
model = YOLO("yolov8s.pt")


results = model.train(

    data="/content/dataset/data.yaml",

    epochs=100,

    imgsz=640,

    batch=16,

    cfg="bhypa.yaml",

    name="gridSearch"

    )

100%|██████████| 131M/131M [00:00<00:00, 229MB/s]


YOLOv8x summary: 209 layers, 68,229,648 parameters, 0 gradients, 258.5 GFLOPs
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.05, cache=False, cfg=bhypa_new.yaml, classes=None, close_mosaic=10, cls=0.7, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=5, deterministic=True, device=None, dfl=0.75, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8x.pt, momentum=0.8, mosaic=0.5, multi_sca

train: Scanning /content/dataset/labels/train.cache... 161 images, 0 backgrounds, 0 corrupt: 100%|██████████| 161/161 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 467.2±53.0 MB/s, size: 96.6 KB)


...

    100/100      12.5G    0.01201      3.035      1.087          1        640: 100%|██████████| 11/11 [00:12<00:00,  1.12s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.53it/s]

                   all         41         93       0.32       0.28      0.254      0.108



100 epochs completed in 0.664 hours.
Optimizer stripped from runs/detect/CobaBestParamNew2/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/CobaBestParamNew2/weights/best.pt, 136.7MB

Validating runs/detect/CobaBestParamNew2/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 68,125,494 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.40it/s]


                   all         41         93      0.365      0.261      0.264      0.112
             Berlubang         25         43      0.361      0.263      0.283     0.0899
                 Retak         28         50       0.37       0.26      0.245      0.134
Speed: 0.3ms preprocess, 27.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/CobaBestParamNew2


# Save Model

In [ ]:
# simpan model
source = '/content/runs/detect/gridSearch'

destination = '/content/drive/MyDrive/Dataset/Project_Data_Science/new_data/Yolov8_Result_02_bhypa'

shutil.copytree(source, destination)

print('Model telah berhasil disalin')

Model telah berhasil disalin


# Test Model

In [ ]:
# Upload gambar
uploaded = files.upload()

filename = next(iter(uploaded))



# Load model
model = YOLO('/content/runs/detect/gridSearch/weights/best.pt')



# Prediksi
results = model.predict(source=filename, save=True, imgsz=640, conf=0.3)



# Cari file hasil terbaru & tampilkan
result_dirs = sorted(glob.glob('runs/detect/*'), key=os.path.getmtime)

if result_dirs:
    latest_dir = result_dirs[-1]

    result_files = glob.glob(os.path.join(latest_dir, '*.jpg'))

    if result_files:
        display(Image(filename=result_files[0]))

        print(f"Hasil disimpan di: {result_files[0]}")

    else:
        print("Gambar hasil tidak ditemukan.")

else:
    print("Folder hasil prediksi tidak ditemukan.")

Saving istockphoto-937868038-640_adpp_is.mp4 to istockphoto-937868038-640_adpp_is.mp4

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/570) /content/istockphoto-937868038-640_adpp_is.mp4: 384x640 (no detections), 11.8ms
video 1/1 (frame 2/570) /content/istockphoto-937868038-640_adpp_is.mp4: 384x640 1 Retak, 10.7ms
video 1/1 (frame 3/570) /content/istockphoto-937868038-640_adpp_is.mp4: 384x640 1 Retak, 10.8ms
video 1/1 (frame 4/570) /content/istockphoto-937868038-640_adpp_is.mp4: 38